In [13]:
import boto3
import json
from decimal import Decimal

class DecimalEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            # Convert Decimal to float or int
            return float(obj)
        return super(DecimalEncoder, self).default(obj)

dynamodb = boto3.resource('dynamodb', region_name='ap-south-1')

table = dynamodb.Table('Employee')

def export_table_to_json(file_name):
    response = table.scan()
    items = response.get('Items', [])

    with open(file_name, 'w') as file:
        json.dump(items, file, indent=4, cls = DecimalEncoder)
    print(f"Data exported to {file_name}")

export_table_to_json("employee_export.json")


Data exported to employee_export.json


In [22]:
import boto3
import json

from decimal import Decimal


dynamodb = boto3.resource('dynamodb', region_name='ap-south-1')
table = dynamodb.Table('Employee')

def import_json_to_table(file_name):
    with open(file_name, 'r') as file:
        items = json.load(file)

    for item in items:
        item['id'] = Decimal(str(item['id']))  # Convert id to Decimal
        item['age'] = Decimal(str(item['age']))  # Convert age to Decimal

    with table.batch_writer() as batch:
        for item in items:
            batch.put_item(Item=item)
        print(f"Data imported from {file_name}")

# Example Usage
import_json_to_table("employee_import.json")



Data imported from employee_import.json


In [15]:
import json

# Load the JSON file and sanitize it
file_path = 'employee_export.json'
try:
    with open(file_path, 'r') as file:
        # Attempt to load JSON
        data = json.load(file)
    print("JSON loaded successfully:", data)
except json.JSONDecodeError:
    print("The file contains invalid JSON. Please correct the syntax.")


JSON loaded successfully: [{'id': 3.0, 'name': 'Dilip', 'gender': 'M', 'age': 23.0}, {'id': 2.0, 'name': 'Dhananjay', 'gender': 'M', 'age': 24.0}, {'id': 4.0, 'name': 'Ankitha', 'gender': 'F', 'age': 22.0}, {'id': 1.0, 'name': 'Sudhanshu', 'gender': 'M', 'age': 25.0}]
